In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import pyxatu
import numpy as np

xatu = pyxatu.PyXatu()
pd.set_option('display.max_colwidth', None)

color_palette = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf" 
]

In [ ]:
a = pd.read_parquet("/mnt/hdd1/results/slot_usage.parquet")
a["block_number"] = a["block_number"].apply(lambda x: x.tolist())
a['tx_hashes'] = [[item.split('|')[0] for item in row] for row in a['block_number']]
a['block_numbers'] = [[int(item.split('|')[1]) for item in row] for row in a['block_number']]
a.drop("block_number", axis=1, inplace=True)

In [ ]:
latest_slot = xatu.get_slots(columns="max(slot) as slot", add_missed=False)["slot"][0]
txsinfo = xatu.get_slots(
    slot=[10_000_000, int(latest_slot)], 
    columns="execution_payload_block_number,slot, execution_payload_transactions_count",
    add_missed=False
)
txsinfo.columns = ["block_number", "slot", "txcount"]

In [ ]:
unique_blocks = {block for blocks in a["block_numbers"] for block in blocks}
txsinfo = txsinfo[(txsinfo["block_number"].isin(unique_blocks))]
unique_tx_l = int(txsinfo["txcount"].sum())
print(len(unique_blocks))
a["l_bl"] = a.block_numbers.apply(lambda x: len(x))
a["l_tx"] = a.tx_hashes.apply(lambda x: len(x))
a["l_bl"] = a["l_bl"] / len(unique_blocks)
a["l_tx"] = a["l_tx"] / unique_tx_l
a.drop("block_numbers", axis=1, inplace=True)
a.drop("tx_hashes", axis=1, inplace=True)
a = a.sort_values("l_tx", ascending=False).reset_index().iloc[0:100]
a.columns = ["slot", "l_bl", "l_tx"]
a["address"] = 0
mask = a["slot"].str.contains("\|")
a.loc[~mask, "address"] = a.loc[~mask].slot
a.loc[mask, "address"] = a.loc[mask, "slot"].str.split(r"\|").str[0]
a.loc[mask, "storage_key"] = a.loc[mask, "slot"].str.split(r"\|").str[1]
a.drop("slot", axis=1, inplace=True)
a["address"] = a["address"].apply(lambda x: x if len(x) == 42 else "0x" + "0"*(42-len(x)) + x[2:])

In [ ]:
df = a.iloc[:50].copy()

df["address"] = df["address"].apply(lambda x: x[:4] + "..." + x[-2:])
df["storage_key"] = df["storage_key"].apply(lambda x: x[:4] + "..." + x[-2:] if pd.notna(x) and len(x) > 6 else x)

df["l_bl"] = df["l_bl"].round(2)
df["l_tx"] = df["l_tx"].round(2)

df["storage_key"] = df["storage_key"].fillna("")

df.rename(columns={
    "l_bl": "Accesses per Block",
    "l_tx": "Accesses per Transaction",
    "address": "Abbreviated Address",
    "storage_key": "Storage Key",
}, inplace=True)

df = df[["Abbreviated Address", "Storage Key", "Accesses per Block", "Accesses per Transaction"]]
df

In [ ]:
df = pd.read_parquet("/mnt/hdd1/results/storage_epoch_roll.parquet")
if "slot" in df.columns.tolist():
    df.dropna(subset="slot", inplace=True)
    df["date"] = df.slot.apply(lambda x: xatu.helpers.slot_to_time(x))

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.ceil('H')

agg_dict = {
    "savings": ("savings", "sum"),
    "savings_epoch": ("savings_epoch", "sum"),
    "full_saving": ("full_saving", "sum"),
    "gas_used": ("gas_used", "sum"),
    "block_count": ("gas_used", "count")
}

for i in range(1, 32):
    agg_dict[f"savings_roll_{i}"] = (f"savings_roll_{i}", "sum")

df = df.groupby("date").agg(**agg_dict).reset_index()

df["percentage_of_total"] = df["savings"] / df["gas_used"] * 100
df["percentage_of_total_epoch"] = df["savings_epoch"] / df["gas_used"] * 100
df["percentage_of_total_full_saving"] = df["full_saving"] / df["gas_used"] * 100

for i in range(1,32):
    df[f"percentage_of_total_roll_{i}"] = df[f"savings_roll_{i}"] / df["gas_used"] * 100
    df[f"savings_roll_{i}"] = df[f"savings_roll_{i}"] / df["block_count"]    
    
df["gas_used"] = df["gas_used"] / df["block_count"]
df["savings"] = df["savings"] / df["block_count"]
df["savings_epoch"] = df["savings_epoch"] / df["block_count"]
df["full_saving"] = df["full_saving"] / df["block_count"]

In [ ]:
df = df[df["date"] != df.date.max()]

In [ ]:
df.sort_values("date", inplace=True)

df['gas_used_minus_savings'] = df['gas_used'] - df['savings']
df['gas_used_minus_savings_epoch'] = df['gas_used'] - df['savings_epoch']
df['gas_used_minus_savings_roll_5'] = df['gas_used'] - df['savings_roll_5']
df['gas_used_minus_savings_roll_10'] = df['gas_used'] - df['savings_roll_10']
df['gas_used_minus_savings_roll_15'] = df['gas_used'] - df['savings_roll_15']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['gas_used'],
    mode='lines',
    name='Gas Used (currently)',
    line=dict(width=2, color=color_palette[0]),
))

fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['gas_used_minus_savings'],
    mode='lines',
    name='Gas Used with block-level slot warming',
    line=dict(width=2, color=color_palette[1]),
))
fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['gas_used_minus_savings_roll_5'],
    mode='lines',
    name='Gas Used with 5-block warming window',
    line=dict(width=2, color=color_palette[2]),
))

fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['gas_used_minus_savings_roll_10'],
    mode='lines',
    name='Gas Used with 10-block warming window',
    line=dict(width=2, color=color_palette[3]),
))

fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['gas_used_minus_savings_roll_15'],
    mode='lines',
    name='Gas Used with 15-block warming window',
    line=dict(width=2, color=color_palette[4]),
))

fig.update_layout(
    title=dict(
        text='(Multi-) Block-Level Warming',
        font=dict(family="Ubuntu Mono", size=24),
        x=0.5,
        y=0.95
    ),
    xaxis=dict(
        title=None,
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True
    ),
    yaxis=dict(
        title='Gas',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True,
        type="log"
    ),
    legend=dict(
        title_font=dict(family="Ubuntu Mono", size=14),
        font=dict(family="Ubuntu Mono", size=12),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(128, 128, 128, 0.2)',
        borderwidth=1,
            orientation="h",

        x=0.85,
        y=-0.35,
        xanchor='right', 
        yanchor='bottom'
    ),
    template='plotly_white',
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=50, r=20, b=140, l=60),
    showlegend=True,
    legend_title=None,
    hovermode='x unified',
    width=900
)

fig.update_traces(
    hovertemplate='<b>Date</b>: %{x}<br>' +
                  '<b>Gas</b>: %{y:.1f}M<extra></extra>'
)

fig.write_image("save_over_time.png")
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['percentage_of_total']*-1,
    mode='lines',
    name='Block-Warming',
    fill='tozeroy',
    line=dict(width=0),
    fillcolor=hex_to_rgba(color_palette[0],0.8),
    fillpattern=dict(fgcolor="#101010", shape="/"),
    
))

fig.add_trace(go.Scatter(
    x=df['date'],
    y=df['percentage_of_total_roll_15']*-1,
    mode='lines',
    name='Multi-Block-Warming (n=15)',
    fill='tonexty',
    line=dict(width=0),
    fillcolor=hex_to_rgba(color_palette[1],0.8),
    fillpattern=dict(fgcolor="#101010", shape="/"),
    
))
fig.add_shape(
    type="line",
    x0=0,
    x1=1,
    y0=0,
    y1=0,
    xref='paper',
    line=dict(
        color="rgba(128, 128, 128, 0.7)",
        dash="dash",
    )
)

fig.update_layout(
    title=dict(
        text='Savings in Percentage of Block Gas Used',
        font=dict(family="Ubuntu Mono", size=24),
        x=0.5,
        y=0.95
    ),
    xaxis=dict(
        title='Date',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True
    ),
    yaxis=dict(
        title='Gas',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True,
        tickvals=list(range(-18, 5, 2)),
        ticktext=[str(i) + "%"  for i in list(range(-18, 5, 2))],
        range=[-20,5]
    ),
    legend=dict(
        title_font=dict(family="Ubuntu Mono", size=14),
        font=dict(family="Ubuntu Mono", size=12),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(128, 128, 128, 0.2)',
        borderwidth=1,
        x=1,
        y=0.82,
        xanchor='right',
        yanchor='bottom'
    ),
    template='plotly_white',
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=80, r=20, b=40, l=60),
    showlegend=True,
    legend_title=None,
    hovermode='x unified'
)

fig.write_image("perc_delta.png")
fig.show()

In [ ]:
def hex_to_rgba(hex_color: str, alpha: float) -> tuple:
    hex_color = hex_color.lstrip('#')
    if len(hex_color) == 6:
        r, g, b = int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)
    else:
        raise ValueError("Hex color must be 6 characters long.")
    return f"rgba({r}, {g}, {b}, {alpha})"

In [ ]:
values = []
for i in df:
    if not "percentage" in i or not "roll" in i:
        continue
        
    else:        
        values.append(df[i].median())
        
print(values)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1,32)),
    y=values,
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(size=6),
    name='Warming Values'
))

fig.add_shape(
    type="line",
    x0=0,
    x1=35,
    y0=df["percentage_of_total_full_saving"].median(),
    y1=df["percentage_of_total_full_saving"].median(),
    #xref='paper',
    line=dict(
        color="rgba(128, 128, 128, 0.7)",
        dash="dash",
    )
)

fig.add_annotation(
    text="Max saving if everything was warm",
    x=18,
    y=35,
    showarrow=False,
    yshift=0,
    bgcolor="rgba(255, 255, 255, 0.8)",
    font=dict(family="Ubuntu Mono", size=12)
)

# Update layout
fig.update_layout(
    title=dict(
        text='(Multi-) Block-Level Warming Potential',
        font=dict(family="Ubuntu Mono", size=24),
        x=0.5,
        y=0.95
    ),
    xaxis=dict(
        title='Size of Window in Blocks',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True
    ),
    yaxis=dict(
        title='% Savings In Relation to Gas Used',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True,
        tickformat='.2f',
        range=[0,40]
    ),
    legend=dict(
        title_font=dict(family="Ubuntu Mono", size=14),
        font=dict(family="Ubuntu Mono", size=12),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(128, 128, 128, 0.2)',
        borderwidth=1,
        x=0.98,
        y=0.02,
        xanchor='right',
        yanchor='bottom'
    ),
    template='plotly_white',
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=80, r=20, b=40, l=60),
    showlegend=False,
    hovermode='x unified'
)

fig.show()  
fig.write_image("slot_warming_potential_2.png")

In [ ]:
values = []
for i in df:
    if not "percentage" in i or not "roll" in i:
        continue
        
    else:        
        values.append(df[i].mean())
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1,32)),
    y=values,
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(size=6),
    name='Warming Values'
))

fig.update_layout(
    title=dict(
        text='(Multi-) Block-Level Warming Potential',
        font=dict(family="Ubuntu Mono", size=24),
        x=0.5,
        y=0.95
    ),
    xaxis=dict(
        title='Size of Window in Blocks',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True
    ),
    yaxis=dict(
        title='% Savings In Relation to Gas Used',
        title_font=dict(family="Ubuntu Mono", size=14),
        tickfont=dict(family="Ubuntu Mono", size=12),
        gridcolor='rgba(128, 128, 128, 0.2)',
        showgrid=True,
        tickformat='.2f',
        #range=[0,35]
    ),
    legend=dict(
        title_font=dict(family="Ubuntu Mono", size=14),
        font=dict(family="Ubuntu Mono", size=12),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(128, 128, 128, 0.2)',
        borderwidth=1,
        x=0.98,
        y=0.02,
        xanchor='right',
        yanchor='bottom'
    ),
    template='plotly_white',
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=80, r=20, b=40, l=60),
    showlegend=False,
    hovermode='x unified'
)

fig.show()  
fig.write_image("slot_warming_potential.png")